In [1]:
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.neural_network import MLPClassifier

# Load training and testing data
training = pd.read_csv('Data/Training.csv')
testing = pd.read_csv('Data/Testing.csv')
cols = training.columns[:-1]  # Features
x = training[cols]
y = training['prognosis']
y1 = y

# Reduced data for disease-wise maximum symptoms
reduced_data = training.groupby(training['prognosis']).max()

# Label encoding for prognosis
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = le.transform(testing['prognosis'])

# Decision Tree Classifier
clf1 = DecisionTreeClassifier()
clf = clf1.fit(x_train, y_train)
dt_scores = cross_val_score(clf, x_test, y_test, cv=3)

# Neural Network (ANN) Classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
mlp.fit(x_train, y_train)
nn_scores = cross_val_score(mlp, x_test, y_test, cv=3)

# Combined Model using predictions from both classifiers
def combined_model_predict(input_data):
    dt_prediction = clf.predict(input_data)
    nn_prediction = mlp.predict(input_data)
    # Use a simple majority voting or any other strategy
    combined_prediction = dt_prediction if dt_prediction == nn_prediction else nn_prediction
    return combined_prediction

# Combined accuracy for CART-ANN
combined_scores = (dt_scores + nn_scores) / 2
print("CART-ANN Combined Accuracy:", combined_scores.mean())

# Feature importance
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

# Text-to-speech initialization
def readn(nstr):
    engine = pyttsx3.init()
    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)
    engine.say(nstr)
    engine.runAndWait()
    engine.stop()

# Dictionaries for symptoms severity, description, and precautions
severityDictionary = dict()
description_list = dict()
precautionDictionary = dict()
symptoms_dict = {symptom: index for index, symptom in enumerate(x)}

# Load symptom descriptions
def getDescription():
    global description_list
    with open('MasterData/symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if len(row) >= 2:
                _description = {row[0]: row[1]}
                description_list.update(_description)

# Load symptoms severity
def getSeverityDict():
    global severityDictionary
    with open('MasterData/symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if len(row) >= 2:
                _diction = {row[0]: int(row[1])}
                severityDictionary.update(_diction)

# Load precautions
def getprecautionDict():
    global precautionDictionary
    with open('MasterData/symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if len(row) >= 5:
                _prec = {row[0]: [row[1], row[2], row[3], row[4]]}
                precautionDictionary.update(_prec)

# Initial information gathering
def getInfo():
    print("Please enter your name: ")
    print("कृपया अपना नाम बताएं:", end="->")
    name = input("astitva")
    print("Hello", name)

# Pattern checking for symptoms input
def check_pattern(dis_list, inp):
    import re
    pred_list = []
    ptr = 0
    patt = "^" + inp + "$"
    regexp = re.compile(inp)
    for item in dis_list:
        if regexp.search(item):
            pred_list.append(item)
    if len(pred_list) > 0:
        return 1, pred_list
    else:
        return ptr, item

# Secondary prediction using the decision tree
def sec_predict(symptoms_exp):
    df = pd.read_csv('Data/Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[symptoms_dict[item]] = 1
    
    return rf_clf.predict([input_vector])

# Print the disease prediction
def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return disease

# Convert the decision tree to code
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []

    print("Enter the symptoms you are experiencing (separated by commas): ")
    disease_input = input("").split(",")

    for symptom in disease_input:
        conf, cnf_dis = check_pattern(chk_dis, symptom.strip())
        if conf == 1:
            symptoms_present.append(cnf_dis[0])
        else:
            print(f"Invalid symptom: {symptom.strip()}. Please enter a valid symptom.")

    num_days = int(input("Okay. From how many days? अच्छा । कितने दिनों से? : "))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name in symptoms_present:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("Are you experiencing any of the following symptoms?")
            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                print(syms, "? : ", end='')
                while True:
                    inp = input("")
                    if inp == "yes" or inp == "no":
                        break
                    else:
                        print("Provide proper answers i.e. (yes/no) : ")
                if inp == "yes":
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:
                print("You may have", present_disease[0])
                print(description_list[present_disease[0]])
            else:
                print("You may have", present_disease[0], "or", second_prediction[0])
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])

            precution_list = precautionDictionary[present_disease[0]]
            print("Take the following measures:")
            for i, j in enumerate(precution_list):
                print(i + 1, ")", j)

            confidence_level = (1.0 * len(symptoms_present)) / len(symptoms_given)
            print("Confidence level is " + str(confidence_level))

    recurse(0, 1)

# Calculate condition severity
def calc_condition(exp, days):
    sum_severity = 0
    for item in exp:
        if item in severityDictionary:
            sum_severity += severityDictionary[item]
        else:
            # Default severity if symptom is missing
            sum_severity += 5
    severity_score = (sum_severity * days) / (len(exp) + 1)
    if severity_score > 13:
        print("You should take the consultation from a doctor.")
    else:
        print("Your condition seems manageable at home, but monitor your symptoms closely.")
    print("Severity score:", severity_score)

# Load dictionaries
getSeverityDict()
getDescription()
getprecautionDict()
getInfo()
tree_to_code(clf, cols) 


CART-ANN Combined Accuracy: 0.9870700925123854
Please enter your name: 
कृपया अपना नाम बताएं:->Hello astitva
Enter the symptoms you are experiencing (separated by commas): 
Are you experiencing any of the following symptoms?
itching ? : skin_rash ? : stomach_pain ? : burning_micturition ? : spotting_ urination ? : Your condition seems manageable at home, but monitor your symptoms closely.
Severity score: 9.0
You may have Drug Reaction
An adverse drug reaction (ADR) is an injury caused by taking medication. ADRs may occur following a single dose or prolonged administration of a drug or result from the combination of two or more drugs.
Take the following measures:
1 ) stop irritation
2 ) consult nearest hospital
3 ) stop taking drug
4 ) follow up
Confidence level is 0.4


c:\Users\astit\Desktop\vscodes\hcbipynb\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
